# 🎥 Real-ESRGAN Video Upscaler
This notebook allows you to upload a low-quality video, enhance it using Real-ESRGAN, and download the final version with audio preserved.

In [ ]:
# 📥 Download video from a provided URL (passed via ?video_url=...)
import os, requests
from urllib.parse import unquote
from IPython.display import display, HTML

# Get video_url from Colab params
try:
    from google.colab import output
    from IPython import get_ipython
    ipython = get_ipython()
    video_url = ipython.getoutput("echo $video_url")[0] if 'video_url' in os.environ else ''
except:
    video_url = ''

if video_url:
    video_url = unquote(video_url)
    print(f"Downloading from: {video_url}")
    response = requests.get(video_url, stream=True)
    with open("input.mp4", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("✅ Downloaded video as input.mp4")
else:
    print("⚠️ No video URL provided. Upload manually below if needed.")

In [ ]:
#@title 🛠️ Setup
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip -q install basicsr facexlib gfpgan
!pip -q install -r requirements.txt
!python setup.py develop
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install "numpy<2.0"


In [ ]:
#@title 📤 Upload Video
from google.colab import files
import shutil
uploaded = files.upload()
video_path = "input.mp4"
for name in uploaded.keys():
    shutil.move(name, video_path)
print(f"✅ Uploaded video saved as {video_path}")


In [ ]:
#@title 🖼️ Extract Frames
import cv2, os
frames_dir = 'input_frames'
os.makedirs(frames_dir, exist_ok=True)
cap = cv2.VideoCapture(video_path)
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(frames_dir, f"{frame_count:05d}.png")
    cv2.imwrite(frame_path, frame)
    frame_count += 1
cap.release()
print(f"🖼️ Extracted {frame_count} frames to '{frames_dir}'")


> ✅ Now manually run Real-ESRGAN on extracted frames OR set output folder below.

In [ ]:
#@title 🚀 Enhance Frames using Real-ESRGAN
!python inference_realesrgan.py -n RealESRGAN_x4plus -i input_frames -o results


In [ ]:
#@title 🧪 Combine Upscaled Frames
import glob
output_folder = 'results'
upscaled_frames = sorted(glob.glob(os.path.join(output_folder, '*.png')))
if not upscaled_frames:
    raise ValueError("❌ No upscaled frames found!")
first_frame = cv2.imread(upscaled_frames[0])
height, width, _ = first_frame.shape
upscaled_video_path = 'upscaled_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 24
out = cv2.VideoWriter(upscaled_video_path, fourcc, fps, (width, height))
for frame_path in upscaled_frames:
    frame = cv2.imread(frame_path)
    out.write(frame)
out.release()
print(f"🎉 Video saved to: {upscaled_video_path}")


In [ ]:
#@title 🔊 Merge with Original Audio
final_output = "final_upscaled_with_audio.mp4"
!ffmpeg -y -i "{upscaled_video_path}" -i "{video_path}" -c copy -map 0:v:0 -map 1:a:0 -shortest "{final_output}"


In [ ]:
# ✅ Final UI "Popup" Style Section
from IPython.display import display, HTML

html_popup = f"""
<div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 20px; background-color: #f9fff9; box-shadow: 0 4px 12px rgba(0,0,0,0.1); max-width: 700px; margin: 20px auto; font-family: sans-serif;">
  <h2 style="color: #2e7d32;">✅ Enhancement Complete!</h2>
  <p>Your enhanced video is ready. Preview it below or click to download:</p>
  <video width="100%" controls style="border-radius: 8px;">
    <source src="final_output.mp4" type="video/mp4">
    Your browser does not support the video tag.
  </video>
  <br><br>
  <a href="final_output.mp4" download style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-size: 16px;">
    📥 Download Enhanced Video
  </a>
</div>
"""

display(HTML(html_popup))


In [ ]:
#@title 📥 Download Final Video
from google.colab import files
files.download("final_upscaled_with_audio.mp4")
